# Relatorio Técnico Referente a Matéria de Técnicas de Pesquisa Experimental
  
**Tema:** Analise da Variação de Parâmetros na Impressão 3D de Filamentos Poliméricos  
**Profº:** Pedro Bastos Costa  
**Universidade Federal de Minas Gerais - UFMG**

**Membros:**
* Guilherme de Paula Rúbio  
* Matheus Ungaretti Borges  
* Daniel Affonso Vasconcelos  

## *Importação de Bibliotecas*

In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as st 
import scipy.linalg as ln
import matplotlib.pyplot as plt 

from math import sqrt

%matplotlib inline

## ***Fuções Auxiliares Criadas***  

* **Função**: Teste de hipótese bicaudal, distribuição F

In [2]:
def F_bicaudal_hypoteses (F :float, alpha :float, dn :float, dd :float):
    '''
        Função de teste de hipótese de um teste bicaudal do tipo F
            Entrada:
               * F: Valor calculado de F_0;
               * alpha: nivel de confiança total;
               * dn: graus de liberdade do numerador;
               * dd: graus de liberdade do denominado.

            Saídas:
               * Impressão das Respostas.

            Retorno: Nenhum
    '''
    min = st.f.ppf(alpha/2, dn, dd)         # Limite Superior
    max = st.f.ppf(1-(alpha/2), dn, dd)     # Limite Inferior

    print('O valor da estatísca cálcula F0 é: {:.4f}'.format(F))
    print('O limites da distribuição para {} de {:.2f} é: [{:.4f}:{:.4f}]'.format('\u0251', alpha, min, max))
    print('Portanto,',end=' ')

    if (F<min) or (F>max):
        print('\033[33mREJEITA-SE\033[m a hipótese nula!')
        print('p-valor: {}'.format( st.f.cdf(F, dn, dd) if (F<min) else (1-st.f.cdf(F, dn, dd)) ))
    else:
        print('\033[31mFALHA EM REJEITAR\033[m a hipótese nula!')

   * **Função:** Calculo do erro padrão de uma regressão linear multipla

In [3]:
def lin_mult_regress_error (C0, var2):
    '''
    Função de calculo de erro padrão de cada constantes:
        Entradas:
            * C0 = Matriz de covariância da regressão
            * var2 = variancia do erro padrão
        Saida/Retorno:
            * se = array com os erros padrões
    '''
    se = []

    for i in range(len(C0)):
        se.append(sqrt(var2*C0[i,i]))
    
    return se

## *Importação dos dados*
Todos os dados utilizados nessa análise estão disponibilizados na plataforma Kaggle. Caso queria acessar a fonte dos dados utilizados basta [clicar aqui.](https://www.kaggle.com/afumetto/3dprinter?select=data.csv)  

### Contexto  
Os dados utilizados são oriundos de uma pesquisa desenvolvida pelo Departamento de Engenharia Mecânica da TR/Selcuk University.  
Essa pesquisa visava avaliar quais os parâmetros de impressão interferem na qualidade de impressão de peças, precisão e rigidez.  
Nos dados apresentados existem nove parâmetro de configuração (entradas), e as medições de três parâmetros de saída.  

### Materiais e Métodos  
* **Impressora:** Ultimaker S5 3-D 
* **Teste de materiais e resistência:** Sincotec GMBH, capacidade de tração - 20kN.

### Conteúdo dos dados
#### Entradas - Parâmetros de configuração:  
* Altura de camada (*Layer Height*) \[mm]  
* Espessura de casca (*Wall Thickness*) \[mm]  
* Densidade de preenchimento (*Infill Density*) \[%]  
* Padrão de preenchimento (*Infill Pattern*)  
* Temperatura do bico de extrusão (*Nozzle Temperature*) \[°C]  
* Temperetura da mesa de impressão (*Bed Temperature*) \[°C]  
* Velocidade de impressão (*Print Speed*) \[mm/s]  
* Material (*Material*)  
* Velocidade do vetilador da extrusora (*Fan Speed*) \[%]  

#### Saídas  
* Rugosidade (*Roughness*) \[µm]
* Tensão de ruptura (*Ultimate Tension Strenght*) \[MPa]
* Alongação (*Elongation*) \[%]  

Abaixo importamos os dados para a análise.  

In [4]:
dados = pd.read_csv('datasets\data_3D_print.csv')   # Importa o DataFrame
dados.head()     # Apresenta as 5 primeiras linhas do DataFrame

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,roughness,tension_strenght,elongation
0,0.02,8,90,grid,220,60,40,abs,0,25,18,1.2
1,0.02,7,90,honeycomb,225,65,40,abs,25,32,16,1.4
2,0.02,1,80,grid,230,70,40,abs,50,40,8,0.8
3,0.02,4,70,honeycomb,240,75,40,abs,75,68,10,0.5
4,0.02,6,90,grid,250,80,40,abs,100,92,5,0.7


## *Análise das Saídas*  
Como temos três saidas iremos analisa-las separadamente.  

### Saida Rugosidade:  

#### ***Influência do material***  
Umas das primeiras hipóteses que desejamos saber do ponto de vista da rugosidade é se o material afeta estatisticamente seu valor, para isso separaremos as saídas de rugosidade por material para realizar uma análise de experimento com um único fator.

In [5]:
dados_roughness_material = dados[['material','roughness']]
dados_roughness_material.head()

,material,roughness
0,abs,25
1,abs,32
2,abs,40
3,abs,68
4,abs,92


Com os dados separados podemos observar que nosso problema é dividido em dois materiais, ou seja, dois níveis (PLA e ABS). Primeiramente vamos analisar quantas observações de cada materiais nos temos.  

In [6]:
dados_roughness_material['material'].value_counts()

abs    25
pla    25
Name: material, dtype: int64

Como podemos observa o número de observações de cada material é igual.
***
#### Teoria 📖:  
Supomos que nosso valor de saída rugosidade varia de acordo com o material utilizado, o que significaria que ela teria o seguinte comportamento:  
$$
\mathsf{
    Y_{ij}= \mu + \tau_{i} + \epsilon_{ij}
}
\left\{\displaystyle
    \begin{array}{l}
        \mathsf{i=0, 1}\\
        \mathsf{j=0, 1, 2, 3, \dots, 25}
    \end{array}
\right.
$$  
Onde:
   * $\mu$: média global;  
   * $\tau_{i}$: fator de alteração referente ao material utilizado;
   * $\epsilon_{ij}$: erro aleatorio da medição  
***  
Para garantirmos que o realmente o material interfere na rugosidade medida nas nossas peças temos que garantir que o valor de $\tau$ de cada material seja diferente de zero. Logo temos que fazer um teste assumindo as seguintes hipóteses:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Para isso faremos um teste ANOVA. Para isso assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Para iniciarmos os calculos vamos primeiramente calular as média e armazena-las

In [7]:
mean_roug_mat = dados_roughness_material.groupby('material').mean()   # media da rugosidade por material
mean_roug_mat

,roughness
material,
abs,193.44
pla,147.72


In [8]:
mean_roug_mat_gen = dados_roughness_material['roughness'].mean()  # Média Geral 
mean_roug_mat_gen

170.58

Calculando o $SQ_{tratamentos}$:

In [9]:
n = (dados_roughness_material['material'].value_counts()).loc['abs']
SQ_trat = n * sum((mean_roug_mat['roughness']-mean_roug_mat_gen)**2)
SQ_trat

26128.980000000003

Calculando $SQ_E$

In [10]:
SQ_abs = sum((dados_roughness_material[dados_roughness_material['material']=='abs']['roughness']-float(mean_roug_mat.loc['abs']))**2)
SQ_pla = sum((dados_roughness_material[dados_roughness_material['material']=='pla']['roughness']-float(mean_roug_mat.loc['pla']))**2)

SQ_e = SQ_abs+SQ_pla
SQ_e

454451.19999999995

Com os erros quadráticos calculados podemos fazer o teste de hipótese:  

In [11]:
a = len(mean_roug_mat)

MQ_trat = SQ_trat/(a-1)
MQe = SQ_e/(a*(n-1))

F0 = MQ_trat/MQe

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

O valor da estatísca cálcula F0 é: 2.7598
O limites da distribuição para ɑ de 0.05 é: [0.0010:5.3541]
Portanto, FALHA EM REJEITAR a hipótese nula!


Como podemos observar para um $\alpha$ de 0,05 FALHAMOS EM REJEITAR a hipótese nula, logo o material não afeta estatísticamente no valor de rugosidade.   
Além do teste ANOVA utilizaremos então o método MDS para avaliar se a as médias medidas tem diferença significativas na rugosidade.  
Para isso temos que para as médias terem diferenças significativas devemos relacionar:  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> MDS
}
$$  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> \mathit{t}_{\frac{\alpha}{2};a \cdot(n-1)} \cdot \sqrt{\dfrac{2\cdot MQ_E}{n}}
}
$$  
Assim:  

In [12]:
MDS = st.t.ppf(1-0.025,a*(n-1)) * sqrt((2*MQe)/n)

print('{:^10}|{:^10}'.format('abs-pla','MDS'))
print('{:^1}{:.4f}{:^2}|{:^1}{:.4f}{:^2}'.format('',float(mean_roug_mat.loc['abs'] - mean_roug_mat.loc['pla']),'','', MDS,''))

 abs-pla  |   MDS    
 45.7200  | 55.3352  


Analisando a mínima diferença significativa (MDS), vemos que realmente a mudança do material não afeta estatistiamente o valor da rugosidade.  
Essa conclusão era esperada, uma vez que como a impressão ocorre por camadas, a rugosidade deve ser afeta prinicpalmente por parâmetros que determinam as caracteristicas dessas camadas que o material em si.  

### ***Analise do preenchimento interno***  
Com a influência do material sendo descartada agora avaliaremos se o preenchimento interno da peça influencia na rugosidade da peça.  
Primeiramente vamos separar os dados de preenchimento interno e rugosidade.   

In [13]:
pattern_roughness = dados[['infill_pattern','roughness']]
pattern_roughness.head()

,infill_pattern,roughness
0,grid,25
1,honeycomb,32
2,grid,40
3,honeycomb,68
4,grid,92


In [14]:
pattern_roughness['infill_pattern'].value_counts()

honeycomb    25
grid         25
Name: infill_pattern, dtype: int64

Separado os dados vamos então realizar um teste ANOVA seguindo as hipóteses:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Para isso assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0~=~\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  
Assim calculamos $F_0$ e fazemos o teste de hipótese:  

In [15]:
patt_rough_mean = pattern_roughness.groupby('infill_pattern').mean()
patt_rough_mean

,roughness
infill_pattern,
grid,177.28
honeycomb,163.88


In [16]:
patt_rough_gen = pattern_roughness.mean()
patt_rough_gen

roughness    170.58
dtype: float64

In [17]:
n_ptt = (pattern_roughness['infill_pattern'].value_counts()).loc['grid']
SQ_tra_ptt = n_ptt * sum( (patt_rough_mean['roughness'] - float(patt_rough_gen))**2 )
SQ_tra_ptt

2244.500000000002

In [18]:
SQ_e_grid = sum((pattern_roughness[pattern_roughness['infill_pattern']=='grid']['roughness'] - float(patt_rough_mean.loc['grid']))**2)
SQ_e_hone = sum((pattern_roughness[pattern_roughness['infill_pattern']=='honeycomb']['roughness'] - float(patt_rough_mean.loc['honeycomb']))**2)

SQ_e_ptt = SQ_e_grid + SQ_e_hone
SQ_e_ptt

478335.67999999993

In [19]:
a_ptt = len(patt_rough_mean)

MQ_trat_ptt = SQ_tra_ptt/(a-1)
MQe_ptt = SQ_e_ptt/(a*(n-1))

F0_ptt = MQ_trat_ptt/MQe_ptt

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

O valor da estatísca cálcula F0 é: 2.7598
O limites da distribuição para ɑ de 0.05 é: [0.0010:5.3541]
Portanto, FALHA EM REJEITAR a hipótese nula!


#### ***Modelo de Regressão Linear Multipla***  
Com os dados restantes vamos fazer um modelo de regressão multipla com 7 variáveis $\mathsf{\left(x_1, x_2, x_3, x_4, x_5, x_6, x_7\right)}$, sendo cada uma dessas uma entrada medida restante, e possui 8 coeficientes $\mathsf{\left(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2, \hat{\beta}_3, \hat{\beta}_4, \hat{\beta}_5, \hat{\beta}_6, \hat{\beta}_7\right)}$.  
Primeiramente vamos separar os dados com relação com a rugosidade:  

In [20]:
data_roughness = dados.drop(['infill_pattern', 'material', 'tension_strenght', 'elongation'], axis=1)
data_roughness.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,print_speed,fan_speed,roughness
0,0.02,8,90,220,60,40,0,25
1,0.02,7,90,225,65,40,25,32
2,0.02,1,80,230,70,40,50,40
3,0.02,4,70,240,75,40,75,68
4,0.02,6,90,250,80,40,100,92


***
#### Teoria 📖  
Para determinarmos os coeficiente usaremos a seguinte relação: 
$$
\mathsf{
    \hat{\beta} = \left( X' \cdot X \right)^{-1} \cdot X' \cdot y
}
$$  
Onde:  
$$
\mathsf{y} = \left[
    \begin{array}{c}
        \mathsf{y_1}\\ \mathsf{y_2}\\ \vdots \\ \mathsf{y_n}
    \end{array}
\right];

\mathsf{X} = \left[
    \begin{array}{ccccc}
        1 & \mathsf{x_{11}} & \mathsf{x_{12}} & \cdots & \mathsf{x_{1k}}\\
        1 & \mathsf{x_{21}} & \mathsf{x_{22}} & \cdots & \mathsf{x_{2k}}\\
        \vdots & \vdots & \vdots & \vdots & \vdots\\
        1 & \mathsf{x_{n1}} & \mathsf{x_{n2}} & \cdots & \mathsf{x_{nk}}
    \end{array}
\right]; 

\mathsf{\hat{\beta}} = \left[
    \begin{array}{c}
        \mathsf{\beta_0}\\ \mathsf{\beta_1}\\ \vdots \\ \mathsf{\beta_0}
    \end{array}
\right]
$$  

Desta equação temos a matriz de covariância que é:  
$$
\mathsf{
    C = \left(X' \cdot X \right)^{-1}
}
$$  
***
Primeiramente vamos determinar a matriz de covariância:  

In [21]:
n = len(data_roughness)
X_roug = np.array(pd.concat([pd.Series(np.ones(len(data_roughness))),data_roughness.iloc[:,:7]],axis=1))
Y_roug = np.array(data_roughness['roughness'])

C = np.linalg.inv((X_roug.T).dot(X_roug))
C

array([[-1.37564362e+14,  1.15720408e+00,  7.06375086e-03,
        -1.03045304e-04,  5.03537288e-04,  2.29273937e+12,
         4.17333108e-04, -4.58547874e+11],
       [ 3.34108686e-01,  5.24386052e+00,  3.13062506e-02,
        -3.77559096e-04,  1.21916643e-03, -2.42494531e-02,
         1.89362315e-03,  4.32886846e-03],
       [-3.00018789e-02,  3.13062506e-02,  3.18852006e-03,
        -4.09108343e-05,  1.25746058e-04, -3.81390035e-04,
         1.32146819e-04,  5.20826542e-05],
       [ 7.91752581e-04, -3.77559096e-04, -4.09108343e-05,
         3.57060981e-05, -2.41652827e-05,  4.25007700e-05,
         1.14141170e-06, -2.56327763e-06],
       [-1.45193277e-02,  1.21916643e-03,  1.25746058e-04,
        -2.41652827e-05,  1.65267613e-04, -3.30446443e-04,
         3.39295021e-06,  2.54605757e-05],
       [ 2.29273937e+12, -3.72677448e-02, -9.99376576e-04,
         5.72053087e-05, -5.79531780e-04, -3.82123228e+10,
        -6.57502051e-05,  7.64246457e+09],
       [-2.62781756e-03,  1.893623

Com a matriz de covariância podemos calcular os coeficientes da regressão:  

In [22]:
B_rough = (C.dot(X_roug.T)).dot(Y_roug)

for i in range(len(B_rough)):
    print('{}{}: {:.4f}'.format('\u03B2', i, B_rough[i]))

β0: 372.4155
β1: 770.6541
β2: 1.5708
β3: 0.2734
β4: 1.8998
β5: -13.1164
β6: 0.6373
β7: 3.1169


Assim o modelo de multiplas variáveis dado é:  
$$
\mathsf{
    \hat{y}_{rugosidade} = 372,4155 + 770,6541 \cdot x_1 + 1,5708 \cdot x_2 + 0,2734 \cdot x_3 - 1,8998 \cdot x_4 -13,1164 \cdot x_5 + 0,6373 \cdot x_6 + 3,1169 \cdot x_7
}
$$  

#### Teste para significância  da Regressão  
Para avliar se a nossa regressão é significativa assumimos as hipoteses:  
$$
\left\{
    \begin{array}{l}
        \mathsf{\mathit{H_0:} \hat{\beta}_i= 0}\\
        \mathsf{\mathit{H_1: } \hat{\beta}_j \not= 0}
    \end{array}
\right.
$$  
Com a estatística:  
$$
\mathsf{
    F_0 = \dfrac{\dfrac{SQ_R}{k}}{\dfrac{SQ_E}{n-p}} 
}
$$  
Onde:
$$
\begin{array}{ccc}
    \mathsf{SQ_R = \displaystyle\sum_{i=1}^{n=50} \left(\hat{y}_i - \bar{y}\right)^2} & \mathsf{e} & \mathsf{k = p-1}
\end{array}
$$  
$$
\mathsf{
    SQ_E = \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}
}
$$

In [23]:
Y_hat_roug = B_rough[0] + B_rough[1]*data_roughness['layer_height'] + B_rough[2]*data_roughness['wall_thickness'] + B_rough[3]*data_roughness['infill_density'] + B_rough[4]*data_roughness['nozzle_temperature'] + B_rough[5]*data_roughness['bed_temperature'] + B_rough[6]*data_roughness['print_speed'] + B_rough[7]*data_roughness['fan_speed']

SQe_roug = sum( (Y_roug-Y_hat_roug)**2 )

n_roug = len(Y_roug)
p_roug = len(B_rough)-1
k_roug =  p_roug-1

SQr_roug = sum( (Y_hat_roug - data_roughness['roughness'].mean())**2 )

F0_roug = (SQr_roug/k_roug)/(SQe_roug/(n_roug-p_roug))

F_bicaudal_hypoteses(F0_roug, 0.05, k_roug, n_roug-p_roug)

O valor da estatísca cálcula F0 é: 9.5380
O limites da distribuição para ɑ de 0.05 é: [0.1999:2.7195]
Portanto, REJEITA-SE a hipótese nula!
p-valor: 1.1553901703154068e-06


Como podemos ver falhamos em rejeitar a hipôtese nula, logo ao menos uma constate tem influência estatística no valor de rugosidade.

Agora basta determinarmos quais das variáveis interferem realmente na rugosidade. Para isso consideraremos as seguintes hipóteses para o teste de hipóteses dos coeficientes:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0:~}\hat{\beta}_j = 0}\\
        \mathsf{\mathit{H_1:~}\hat{\beta}_j \not=0}
    \end{array}
\right.
$$  
Porem agora utilizaremos a seguinte estatística:  
$$
\mathsf{
    T_0 = \dfrac{\hat{\beta}j - \beta_{j_0}}{\sqrt{\sigma^2 \cdot C_{jj}}} = \dfrac{\hat{\beta}j - \beta_{j_0}}{se(\hat{\beta}_j)}
}
$$ 

Onde: 
$$
\mathsf{
    \hat{\sigma}^2 = \dfrac{SQ_E}{n-p} = \dfrac{\displaystyle \sum_{i=1}^{n=50}{\left(y_i - \hat{y}\right)^2}}{50-7}
}
$$  
Primeiramente calcularemos a variância do erro e os erro padrão de cada coeficiente.  

In [24]:
var = SQe_roug/(n_roug-p_roug)

seB_roug = lin_mult_regress_error(C, var)
seB_roug

ValueError: math domain error

In [25]:
se = []

for i in range(len(C)):
    se.append(float(var*C[i,i]))

se

[-5.18399490577233e+17,
 19761.03825045516,
 12.015664150058617,
 0.13455536565422027,
 0.6227968145232046,
 -143999858493670.97,
 0.10900215410839553,
 -5759994339746.895]

In [26]:
X = pd.concat([pd.Series(np.ones(len(data_roughness))),data_roughness.iloc[:,:7]],axis=1)
X

,0,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,print_speed,fan_speed
0,1.0,0.02,8,90,220,60,40,0
1,1.0,0.02,7,90,225,65,40,25
2,1.0,0.02,1,80,230,70,40,50
3,1.0,0.02,4,70,240,75,40,75
4,1.0,0.02,6,90,250,80,40,100
5,1.0,0.02,10,40,200,60,40,0
6,1.0,0.02,5,10,205,65,40,25
7,1.0,0.02,10,10,210,70,40,50
8,1.0,0.02,9,70,215,75,40,75
9,1.0,0.02,8,40,220,80,40,100


In [28]:
A = X.T @ X
A

,0,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,print_speed,fan_speed
0,50.0,5.300,261.00,2670.0,11075.00,3500.0,3200.0,2500.0
layer_height,5.3,0.765,25.89,283.3,1173.95,371.0,334.0,265.0
wall_thickness,261.0,25.890,1781.00,14310.0,57560.00,18240.0,14920.0,12900.0
infill_density,2670.0,283.300,14310.00,174100.0,595800.00,186900.0,167400.0,133500.0
nozzle_temperature,11075.0,1173.950,57560.00,595800.0,2463875.00,778375.0,708800.0,569375.0
bed_temperature,3500.0,371.000,18240.00,186900.0,778375.00,247500.0,224000.0,187500.0
print_speed,3200.0,334.000,14920.00,167400.0,708800.00,224000.0,248000.0,160000.0
fan_speed,2500.0,265.000,12900.00,133500.0,569375.00,187500.0,160000.0,187500.0


In [31]:
ln.inv(A)

array([[-1.37683989e+14,  1.15976792e+00,  7.14111328e-03,
        -1.06811523e-04,  4.88281250e-04,  2.29473314e+12,
         4.39542826e-04, -4.58946629e+11],
       [ 3.39268857e-01,  5.24386052e+00,  3.13062506e-02,
        -3.77559096e-04,  1.21916643e-03, -2.36354910e-02,
         1.89362315e-03,  4.49744160e-03],
       [-3.04713938e-02,  3.13062506e-02,  3.18852006e-03,
        -4.09108343e-05,  1.25746058e-04, -3.73790832e-04,
         1.32146819e-04,  5.09741000e-05],
       [ 7.86330292e-04, -3.77559096e-04, -4.09108343e-05,
         3.57060981e-05, -2.41652827e-05,  4.23823821e-05,
         1.14141170e-06, -2.53334175e-06],
       [-1.47605066e-02,  1.21916643e-03,  1.25746058e-04,
        -2.41652827e-05,  1.65267613e-04, -3.25131048e-04,
         3.39295021e-06,  2.40110968e-05],
       [ 2.29473314e+12, -3.73104755e-02, -1.00088120e-03,
         5.72204590e-05, -5.79833984e-04, -3.82455524e+10,
        -6.60494165e-05,  7.64911048e+09],
       [-2.66602343e-03,  1.893623

In [ ]:
np.finfo(A.dtype).eps

In [ ]:
np.linalg.cond(A)

In [32]:
B = np.array([[1.0,2.0],[3.0,4.0]])
B

array([[1., 2.],
       [3., 4.]])

In [33]:
ln.inv(B.T @ B)

array([[ 5. , -3.5],
       [-3.5,  2.5]])

In [34]:
(B.T @ B)@ln.inv(B.T @ B)

array([[1.00000000e+00, 7.10542736e-15],
       [0.00000000e+00, 1.00000000e+00]])

In [ ]:
np.linalg.cond(B.T @ B)

In [ ]:
np.linalg.cond(B.T @ B) > np.finfo((B.T @ B).dtype).eps